# INFO6105: Assignment 4 - Making your own Comic Book!

## The notebook follows the given order to create a comic book:

1. Adding text and border to the images
2. Transforming the images into cartoon-type images. These will be the images we see in the comic
3. Horizontally stitch the images
4. Vertically stitch the horizontal strips to form single images
5. Join these single final page images into a PDF

### Install language translation API

In [1]:
!pip install translators
!pip install fpdf
!pip install xlwt
!pip install opencv-python
!pip install xlrd>=2.0.1



zsh:1: 2.0.1 not found


In [2]:
pip install --upgrade pandas xlwt


Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
from IPython.display import Image
from PIL import Image as pili, ImageOps as piliops, ImageDraw as pild, ImageFont as pilf
import cv2
from fpdf import FPDF

#import translators as ts
import translators.server as tss

import time

import pathlib

import math

def translate(text, to_language):
    print(f'to_language: {to_language} => text: {text}')
    #time.sleep(2)
    if to_language == 'en':
        return textx
    
    result = tss.google(text, to_language = to_language)
    #print(result)
    return result

def translate_hindi_csv(source = 'RiseOfMachinesDialogues.xlsx', target = 'RiseOfMachinesDialogues-Combined.xlsx'):
    data = pd.read_excel(source)
    english_dialouges = data['en']
    hindi_dialouges = [translate(dialogue, 'hi') for dialogue in english_dialouges]

    header = ['en', 'hi']
    dialouges = list(zip(english_dialouges, hindi_dialouges))

    dataframe = pd.DataFrame(dialouges, columns= header)

    with pd.ExcelWriter(target) as writer:
        dataframe.to_excel(writer)

def concat_image_horizontal(image_1, image_2):
    result = pili.new('RGB', (image_1.width + image_2.width, image_2.height))
    result.paste(image_1, (0, 0))
    result.paste(image_2, (image_1.width, 0))
    return result

def concat_image_vertical(image_1, image_2):
    result = pili.new('RGB', (image_1.width, image_1.height + image_2.height))
    result.paste(image_1, (0, 0))
    result.paste(image_2, (0, image_1.height))
    return result

def concat_image_horizontal_list(image_list):
    _im = image_list.pop(0)
    for im in image_list:
        _im = concat_image_horizontal(_im, im)
    return _im

def concat_image_vertical_list(image_list):
    _im = image_list.pop(0)
    for im in image_list:
        _im = concat_image_vertical(_im, im)
    return _im

def rmdir(directory):
    for item in directory.iterdir():
        if item.is_dir():
            print(f'Deleting {item}')
            rmdir(item)
        else:
            item.unlink()
    directory.rmdir()

def convert_from_image_to_cv2(img: Image) -> np.ndarray:
    return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

def image_compress(path, ratio):
    image = cv2.imread(path, cv2.IMREAD_UNCHANGED)

    # set the ratio of resized image
    width = int((image.shape[1])/ratio)
    height = int((image.shape[0])/ratio)

    # resize the image by resize() function of openCV library
    return cv2.resize(
        image, 
        (width, height), 
        interpolation = cv2.INTER_AREA)

def cartoonize_poster(
    path, 
    ratio, 
    blur, 
    line, 
    text, 
    number_of_lines = 2, 
    font = 'verdana',
    font_size = 20):

    image = image_compress(path, ratio)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    gray_blur = cv2.medianBlur(gray, blur)
    
    image_with_edges = cv2.adaptiveThreshold(
        gray_blur, 
        255, 
        cv2.ADAPTIVE_THRESH_MEAN_C, 
        cv2.THRESH_BINARY, 
        line, 
        blur)
    
    # display(Image.fromarray(image_with_edges))
    
    # Converting BGR to RGB
    image_with_edges_pil = cv2.cvtColor(image_with_edges, cv2.COLOR_BGR2RGB)

    toon = cv2.bitwise_and(
        image, 
        image, 
        mask = image_with_edges)
    
    if len(text) == 0:
        return toon
    
    
    cblimg_pil = pili.fromarray(cv2.cvtColor(toon, cv2.COLOR_BGR2RGBA))
    
    TINT_COLOR = (0, 0, 0)  # Black
    
    overlay = pili.new('RGBA', cblimg_pil.size, TINT_COLOR+(0,))
    
    #print(f'Using font {font} ...')
    
    '''
    font_to_use = (
        pilf.truetype("ITCKRIST.TTF", 
            24 if ratio == 16 else 18 if ratio == 14 else 18 if ratio == 12 else 20 if ratio == 8 else 82) if font=='ITCKRIST'
        else
            pilf.truetype("Inkfree.ttf", 
                24 if ratio == 16 else 18 if ratio == 14 else 18 if ratio == 12 else 20 if ratio == 8 else 82) if font=='Inkfree'
        else
            pilf.truetype(font + ".ttf", 24 if ratio == 16 else 18 if ratio == 14 else 18 if ratio == 12 else 20 if ratio == 8 else 82)
    )
    '''
    
    #font_name = font + '.ttf'
    font_name = f'{font}.ttf'
    #print(f'font_name: {font_name}')
    
    #font_to_use = ImageFont.truetype(font_name)
    font_to_use = pilf.truetype(font_name, font_size)
    
    draw = pild.Draw(overlay)
    
    #_, h = FONT.getsize(text)
    #_, h = font_to_use.getsize(text)
    h = font_to_use.size
    
    x, y = 0, cblimg_pil.height - (number_of_lines) * h - 900
    
    TRANSPARENCY = .25  # Degree of transparency, 0-100%
    OPACITY = int(255 * TRANSPARENCY)
    
    draw.rectangle(
        (x, y, x + cblimg_pil.width, y + (number_of_lines) * h + 10), 
        fill= TINT_COLOR + (OPACITY,)
    )
    
    fill = (204,0,0)
    draw_dimension = (x + 70, y)
    
    '''
    if ratio == 1:
        draw.text(draw_dimension, text, fill = fill, font = font_to_use) #, stroke_width=1)
    elif ratio < 8:
        draw.text(draw_dimension, text, fill = fill, font = font_to_use)
    else:
        draw.text(draw_dimension, text, fill = fill, font = font_to_use) #, stroke_width=1)
    '''
    
    draw.text(
        draw_dimension, 
        text, 
        fill = fill, 
        font = font_to_use, 
        stroke_width = 1, 
        stroke_fill = 'black')

    cblimg_pil = pili.alpha_composite(cblimg_pil, overlay)
    cblimg_pil = cblimg_pil.convert("RGB")

    return convert_from_image_to_cv2(cblimg_pil)

def cartoonize(raw_img, text, font_file, border_w = 50, border_h = 50, tint_color = (0, 0, 0), opacity = int(255 * 0.5)):
    #add border to image
    img = piliops.expand(raw_img, border=(border_w,border_h), fill='white')

    img_width, img_height = img.size
    font = pilf.truetype(font_file, 3 * int(img_width/100))
        
    #this is dynamic, dialogue lines can vary for images
    num_lines = len(text.split('\n'))
    
    #this is dynamic, sizes vary as per dialogue length
    font_w = font.size
    font_h = font.size

    overlay = pili.new('RGBA', img.size, tint_color+(0,))

    draw = pild.Draw(overlay)

    draw.rectangle(
        (
            border_w, 
            border_h, 
            border_w + img_width - (2*border_w), 
            border_w + (num_lines-0.1*num_lines)*1.15*font_h
        ), 
        fill = tint_color + (opacity,) )
    
    #add text into the dialogue box
    draw.text(
        (border_w,border_h), 
        text, 
        fill = (209,238,9), 
        font = font, 
        align='center')
    
    #alpha composite these two images together to obtain the desired result
    img = pili.alpha_composite(img, overlay)
    return img

def create_comic(min, max, font, language):
    temp_path = pathlib.Path('temp')
    scenes_dir = pathlib.Path('scenes').resolve()
    if temp_path.exists() and temp_path.is_dir():
        print('Removing the temp directory')
        rmdir(temp_path)
    else:
        print('temp directory does not exist')

    # dialogues_file_path = 'KnivesOutScript.xls'
    dialogues_file_path = 'RiseOfMachinesDialogues-combined.xlsx'
    # dialogues_file_path = 'final.csv'

    dialogues = pd.read_excel(dialogues_file_path)
    dialogues = dialogues[language]

    #border width and height that is to be added to each image
    border_w = 50
    border_h = 50
    tint_color = (0, 0, 0) #defining parameters for the text background in the image
    opacity = int(255 * 0.5)
    for i in range(min, max + 1):
        print(f'Parsing scene {i} out of {max}')
        
        #text = dialogues.values[i][0]
        text = dialogues[i - 1]
        
        scene_file_name = f'scenes/Scene-{i}.png'
        # FALLBACK FOR PNG AND JPG

        width = 904
        height = 678
        
        raw_img = pili.open(scene_file_name).convert('RGBA')
        raw_img = raw_img.resize((width, height))
        
        img = cartoonize(raw_img, text, font, border_w, border_h, tint_color, opacity)

        # Useful for debugging the converted scenes
        pathlib.Path('temp/converted-scenes').mkdir(parents=True, exist_ok=True)

        img.convert('RGB').save(f'temp/converted-scenes/scene-{i}.jpeg')
        
        img_raw = cv2.imread(f'temp/converted-scenes/scene-{i}.jpeg')
        
        img_w, img_h = int((img_raw.shape[1])), int((img_raw.shape[0]))
        
        #resive the image
        img_scaled = cv2.resize(img_raw, (img_w, img_h), interpolation=cv2.INTER_AREA)
        
        #convert image to grayscale
        img_gray = cv2.cvtColor(img_scaled, cv2.COLOR_BGR2GRAY)
        
        #contouring on the image
        edges = cv2.Canny(img_gray, 100, 200)
        contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        #contoured image
        cv2.drawContours(img_gray, contours, contourIdx=-1, color=9, thickness=1)
        img_contour = img_gray
        
        #blur the image
        img_blur = cv2.medianBlur(img_contour, 7)
        
        #find edges of the image
        threshold = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 7, 7)
        
        #combine the image using the edge mask
        img_comic = cv2.bitwise_and(img_scaled, img_scaled, mask=threshold)
        
        #convert image to RGB befor saving
        img_comic = cv2.cvtColor(img_comic, cv2.COLOR_BGR2RGB)
        img_comic = pili.fromarray(img_comic)
        
        # Useful for debugging the converted comic scenes
        pathlib.Path('temp/comic-scenes').mkdir(parents=True, exist_ok=True)        
        img_comic = img_comic.convert('RGB').save(f'temp/comic-scenes/scene-{i}.jpeg')

    image_names = [f'temp/comic-scenes/scene-{i}.jpeg' for i in range(min, max + 1)]

    images_per_strip = 5
    strip_per_page = 4

    strip_length = math.ceil(len(image_names)/images_per_strip)
    page_length = math.ceil(strip_length/strip_per_page)

    empty_values = (page_length * strip_per_page * images_per_strip) - len(image_names)
    for _ in range(empty_values):
        image_names.append(None)

    image_names = np.asarray(image_names)
    page_data = image_names.reshape(page_length, strip_per_page, images_per_strip)

    page_index = 0
    strip_index = 0
    pages = []

    for page in page_data:
        print(f'Parsing page {page_index + 1} out of {page_length}')
        horizontal_strips = []

        for strip in page:
            print(f'Parsing horizontal strip {strip_index + 1} out of {strip_length}')

            filtered_strip = list(filter(lambda x : x != None, strip))
            print(f'filtered_strip {filtered_strip}')

            if (len(filtered_strip) != 0):
                images = [pili.open(image) for image in filtered_strip]
                images_comb = concat_image_horizontal_list(images)

                # Useful for debugging the converted horizontal strips
                pathlib.Path('temp/horizontal-strips').mkdir(parents=True, exist_ok=True)
                images_comb.save(f'temp/horizontal-strips/horizontal-strip-{strip_index}.jpeg')

                horizontal_strips.append(images_comb)

            strip_index += 1

        page_image = concat_image_vertical_list(horizontal_strips)

        # Useful for debugging the converted pages
        pathlib.Path('temp/pages').mkdir(parents=True, exist_ok=True)
        image_location = f'temp/pages/page-{page_index}.jpeg'
        page_image.save(image_location)

        pages.append(image_location)

        page_index += 1

    print('Finished parsing')

    # Make the PDF COVER IMAGE
    IMAGE_PATH = 'poster-image.jpg'
    CARTOON_IMAGE_PATH = 'temp/poster-image-cartoonized.jpg'
    
    cover = cartoonize_poster(
        IMAGE_PATH, 
        4, 
        9, 
        11, 
        'Rise of Machines', 
        number_of_lines = 1, 
        font = 'Verdana',
        font_size = 50)
    
     # Converting BGR to RGB
    cover = cv2.cvtColor(cover, cv2.COLOR_BGR2RGB)
    cartoon_image = pili.fromarray(cover)
    
    cartoon_image.save(CARTOON_IMAGE_PATH)

    width, height = cartoon_image.size

    pdf = FPDF(unit = 'pt', format = [width, height])

    pdf.add_page()
    pdf.image(CARTOON_IMAGE_PATH, 0, 0, width, height)

    # Ad rest of the pages in PDF
    for page in pages:
        pdf.add_page()
        pdf.image(page, 0, 0, width, height)

    pdf.output(f'Comic-Rise-Of-Machines-{language}.pdf', 'F')

    print('Finished PDF Generation')

def main():
    translate_hindi_csv()
    create_comic(min = 1, max = 110, font = 'Verdana.ttf', language = 'en')
    create_comic(min = 1, max = 110, font = 'Nirmala.ttf', language = 'hi')

main()

to_language: hi => text:  Moin:  "The future of academia lies in this lab, Srikanth."
to_language: hi => text:  Vinay:  "Indeed, Moin. Our AI robot will change the university experience forever."
to_language: hi => text:  Moin:  "Imagine a campus where students and faculty are always assisted, always connected."
to_language: hi => text:  Vinay (looking at the prototype):  "This... this could be revolutionary."
to_language: hi => text:  Moin:  "It's still a prototype, Vinay. We need more time."
to_language: hi => text:  Vinay:  "Time is money. The world needs to see this now."
to_language: hi => text:  Moin:  "But the programming isn't perfect yet."
to_language: hi => text:  Vinay:  "Perfection comes later. Opportunity knocks now."
to_language: hi => text:  Rohit (to a friend):  "Have you heard? They're unveiling a new AI robot at the event."
to_language: hi => text:  Friend:  "Sounds exciting! Let's go check it out."
to_language: hi => text:  Vinay (at the event):  "Ladies and gentleme

to_language: hi => text:  Moin (reflecting):  "Every setback is a setup for a comeback."
to_language: hi => text:  Moin:  "And we've made quite the comeback, haven't we?"
to_language: hi => text:  Vinay:  "It's been a humbling journey. I've learned the value of patience."
to_language: hi => text:  Rohit:  "As have we all. It's about striking the right balance between ambition and caution."
to_language: hi => text:  Moin:  "We've come full circle. From vision to reality, then crisis to redemption."
to_language: hi => text:  Rohit:  "It's been an incredible journey. And it's only the beginning."
to_language: hi => text:  Vinay:  "I'm committed to ensuring our innovation serves the university and its students."
to_language: hi => text:  Rohit:  "With continued collaboration, we'll achieve wonders."
to_language: hi => text:  Robot (to a group of students):  "Group study session scheduled for 3 PM. Don't be late!"
to_language: hi => text:  Student:  "Thanks! We won't!"
to_language: hi => te

Parsing page 5 out of 6
Parsing horizontal strip 17 out of 22
filtered_strip ['temp/comic-scenes/scene-81.jpeg', 'temp/comic-scenes/scene-82.jpeg', 'temp/comic-scenes/scene-83.jpeg', 'temp/comic-scenes/scene-84.jpeg', 'temp/comic-scenes/scene-85.jpeg']
Parsing horizontal strip 18 out of 22
filtered_strip ['temp/comic-scenes/scene-86.jpeg', 'temp/comic-scenes/scene-87.jpeg', 'temp/comic-scenes/scene-88.jpeg', 'temp/comic-scenes/scene-89.jpeg', 'temp/comic-scenes/scene-90.jpeg']
Parsing horizontal strip 19 out of 22
filtered_strip ['temp/comic-scenes/scene-91.jpeg', 'temp/comic-scenes/scene-92.jpeg', 'temp/comic-scenes/scene-93.jpeg', 'temp/comic-scenes/scene-94.jpeg', 'temp/comic-scenes/scene-95.jpeg']
Parsing horizontal strip 20 out of 22
filtered_strip ['temp/comic-scenes/scene-96.jpeg', 'temp/comic-scenes/scene-97.jpeg', 'temp/comic-scenes/scene-98.jpeg', 'temp/comic-scenes/scene-99.jpeg', 'temp/comic-scenes/scene-100.jpeg']
Parsing page 6 out of 6
Parsing horizontal strip 21 out of 

Parsing page 5 out of 6
Parsing horizontal strip 17 out of 22
filtered_strip ['temp/comic-scenes/scene-81.jpeg', 'temp/comic-scenes/scene-82.jpeg', 'temp/comic-scenes/scene-83.jpeg', 'temp/comic-scenes/scene-84.jpeg', 'temp/comic-scenes/scene-85.jpeg']
Parsing horizontal strip 18 out of 22
filtered_strip ['temp/comic-scenes/scene-86.jpeg', 'temp/comic-scenes/scene-87.jpeg', 'temp/comic-scenes/scene-88.jpeg', 'temp/comic-scenes/scene-89.jpeg', 'temp/comic-scenes/scene-90.jpeg']
Parsing horizontal strip 19 out of 22
filtered_strip ['temp/comic-scenes/scene-91.jpeg', 'temp/comic-scenes/scene-92.jpeg', 'temp/comic-scenes/scene-93.jpeg', 'temp/comic-scenes/scene-94.jpeg', 'temp/comic-scenes/scene-95.jpeg']
Parsing horizontal strip 20 out of 22
filtered_strip ['temp/comic-scenes/scene-96.jpeg', 'temp/comic-scenes/scene-97.jpeg', 'temp/comic-scenes/scene-98.jpeg', 'temp/comic-scenes/scene-99.jpeg', 'temp/comic-scenes/scene-100.jpeg']
Parsing page 6 out of 6
Parsing horizontal strip 21 out of 